# Crunch abrupt-4xCO2 data

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging
import os

import netcdf_scm
import netcdf_scm.crunching

import config

<IPython.core.display.Javascript object>

In [3]:
ID = config.ID

<IPython.core.display.Javascript object>

In [4]:
SOURCE_FILTER = ".*ACCESS.*"
SOURCE_FILTER = ".*"
SOURCE_FILTER

'.*'

<IPython.core.display.Javascript object>

In [5]:
with open("source_filter_abrupt-4xCO2.txt", "w") as fh:
    fh.write(SOURCE_FILTER)

<IPython.core.display.Javascript object>

In [6]:
NETCDF_SCM_LOGGER = logging.getLogger("netcdf_scm")

<IPython.core.display.Javascript object>

In [7]:
STDERR_INFO_HANDLER = logging.StreamHandler()
FORMATTER = logging.Formatter(
    "%(asctime)s %(name)s %(threadName)s - %(levelname)s:  %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
STDERR_INFO_HANDLER.setFormatter(FORMATTER)
STDERR_INFO_HANDLER.setLevel(logging.INFO)

NETCDF_SCM_LOGGER.setLevel(logging.DEBUG)
NETCDF_SCM_LOGGER.addHandler(STDERR_INFO_HANDLER)

<IPython.core.display.Javascript object>

In [8]:
NETCDF_SCM_LOGGER.info("SOURCE_FILTER: %s", SOURCE_FILTER)

2021-07-19 05:39:14 netcdf_scm MainThread - INFO:  SOURCE_FILTER: .*


<IPython.core.display.Javascript object>

In [9]:
netcdf_scm.__version__

'2.1.0'

<IPython.core.display.Javascript object>

In [10]:
!find /data/cmip6/CMIP6/CMIP -mindepth 2 -maxdepth 2 -type d -exec sh -c 'x={};echo $(basename ${x})' \; | sort

ACCESS-CM2
ACCESS-ESM1-5
AWI-CM-1-1-MR
AWI-ESM-1-1-LR
BCC-CSM2-MR
BCC-ESM1
CAMS-CSM1-0
CanESM5
CanESM5-CanOE
CAS-ESM2-0
CESM2
CESM2-FV2
CESM2-WACCM
CESM2-WACCM-FV2
CIESM
CMCC-CM2-HR4
CMCC-CM2-SR5
CMCC-ESM2
CNRM-CM6-1
CNRM-CM6-1-HR
CNRM-ESM2-1
E3SM-1-0
E3SM-1-1
E3SM-1-1-ECA
EC-Earth3
EC-Earth3-AerChem
EC-Earth3-CC
EC-Earth3-LR
EC-Earth3-Veg
EC-Earth3-Veg-LR
FGOALS-f3-L
FGOALS-g3
FIO-ESM-2-0
GFDL-CM4
GFDL-ESM4
GISS-E2-1-G
GISS-E2-1-G-CC
GISS-E2-1-H
GISS-E2-2-G
HadGEM3-GC31-LL
HadGEM3-GC31-MM
IITM-ESM
INM-CM4-8
INM-CM5-0
IPSL-CM5A2-INCA
IPSL-CM6A-LR
IPSL-CM6A-LR-INCA
KACE-1-0-G
KIOST-ESM
MCM-UA-1-0
MIROC6
MIROC-ES2H
MIROC-ES2L
MPI-ESM-1-2-HAM
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MRI-ESM2-0
NESM3
NorCPM1
NorESM1-F
NorESM2-LM
NorESM2-MM
SAM0-UNICON
TaiESM1
UKESM1-0-LL
UKESM1-0-LL


<IPython.core.display.Javascript object>

## Setup

In [11]:
CRUNCH_DIR = "./{}-irf-calibration-crunch".format(ID)
!mkdir -p {CRUNCH_DIR}
CRUNCH_DIR

'./20210416-irf-calibration-crunch'

<IPython.core.display.Javascript object>

## Run

In [12]:
mips = ["CMIP"]

scenarios = [
    "abrupt-4xCO2",
    "piControl"
]
members = [
    "r1i1p1f1",
        "r2i1p1f1",
        "r3i1p1f1",
        "r1i1p1f2",
        "r1i1p2f1",
        "r1i1p1f3",
        "r4i1p1f1",
        "r10i1p1f1",
        "r11i1p1f1",
]
variables = ["tas", "rsdt", "rlut", "rsut"]
tables = ["Amon"]


def get_regexp(inl, trail_slash=True):
    if trail_slash:
        return "({})".format(
            "|".join(["{}{}{}".format(os.sep, s, os.sep) for s in inl])
        )

    return "({})".format("|".join(["{}{}".format(os.sep, s) for s in inl]))


mip_regexp = get_regexp(mips)
scenarios_regexp = get_regexp(scenarios, trail_slash=False)
members_regexp = get_regexp(members, trail_slash=False)
variables_regexp = get_regexp(variables)
tables_regexp = get_regexp(tables, trail_slash=False)

regexp = ".*{}.*{}.*{}.*{}.*{}.*{}.*".format(
    mip_regexp,
    SOURCE_FILTER,
    scenarios_regexp,
    members_regexp,
    tables_regexp,
    variables_regexp,
)
display(regexp)

netcdf_scm.crunching._crunch_data(
    "/data/cmip6/CMIP6/CMIP",
    CRUNCH_DIR,
    "Zebedee Nicholls <zebedee.nicholls@climate-energy-college.org>",
    drs="CMIP6Output",
    regexp=regexp,
    regions="World",
    data_sub_dir="netcdf-scm-crunched",
    force=False,
    small_number_workers=20,
    small_threshold=100,
    medium_number_workers=5,
    medium_threshold=400,
    force_lazy_threshold=600,
    cell_weights=None,
)

'.*(/CMIP/).*.*.*(/abrupt-4xCO2|/piControl).*(/r1i1p1f1|/r2i1p1f1|/r3i1p1f1|/r1i1p1f2|/r1i1p2f1|/r1i1p1f3|/r4i1p1f1|/r10i1p1f1|/r11i1p1f1).*(/Amon).*(/tas/|/rsdt/|/rlut/|/rsut/).*'

2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  netcdf-scm: 2.1.0
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  crunch-contact: Zebedee Nicholls <zebedee.nicholls@climate-energy-college.org>
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  source: /data/cmip6/CMIP6/CMIP
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  destination: ./20210416-irf-calibration-crunch/netcdf-scm-crunched
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  drs: CMIP6Output
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  regexp: .*(/CMIP/).*.*.*(/abrupt-4xCO2|/piControl).*(/r1i1p1f1|/r2i1p1f1|/r3i1p1f1|/r1i1p1f2|/r1i1p2f1|/r1i1p1f3|/r4i1p1f1|/r10i1p1f1|/r11i1p1f1).*(/Amon).*(/tas/|/rsdt/|/rlut/|/rsut/).*
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  regions: World
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  force: False
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  small_number_workers: 20
2021-07-19 05:39:15 netcdf_scm MainThread - INFO:  small_threshold: 100
2021-07-19 05:39:

2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM-1-0/abrupt-4xCO2/r1i1p1f1/Amon/rsut/gr/v20190718
2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM-1-1/piControl/r1i1p1f1/Amon/rsdt/gr/v20201123
2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM-1-1/piControl/r1i1p1f1/Amon/rsdt/gr/v20191029
2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM-1-1/piControl/r1i1p1f1/Amon/rlut/gr/v20201123
2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM-1-1/piControl/r1i1p1f1/Amon/rlut/gr/v20191029
2021-07-19 05:39:16 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/E3SM-Project/E3SM

2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/rlut/gn/v20200630
2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/tas/gn/v20200828
2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/tas/gn/v20190410
2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/tas/gn/v20200630
2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/rsut/gn/v20190715
2021-07-19 05:39:18 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/MOHC/UKESM1-0-LL/piControl/r1i1p1f2/Amon/rsut/gn

2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/rsdt/gr/v20200420
2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/rlut/gr/v20200420
2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/tas/gr/v20200420
2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/rsut/gr/v20200420
2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r1i1p1f1/Amon/rsdt/gr/v20200312
2021-07-19 05:39:19 netcdf_scm.crunching MainThread - INFO:  Adding directory to queue /dat

...

2021-07-19 05:43:48 netcdf_scm.crunching MainThread - ERROR:  Could not calculate size of data in /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-LR/piControl/r1i1p1f1/Amon/rsut/gr/v20190103, exception: Your time_period indicator (199101-199101) does not look right
Traceback (most recent call last):
  File "/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages/netcdf_scm/crunching.py", line 108, in _crunch_data
    helper.load_data_in_directory(dpath, process_warnings=False)
  File "/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages/netcdf_scm/iris_cube_wrappers.py", line 529, in load_data_in_directory
    self._load_and_concatenate_files_in_directory(
  File "/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages/netcdf_scm/iris_cube_wrappers.py", line 1423, in _load_and_concatenate_files_in_directory
    super()._load_and_concatenate_files_in_dire

Sorting directories:  53%|█████▎    | 325/613 [00:35<00:41,  6.91it/s]2021-07-19 05:44:42 netcdf_scm.iris_cube_wrappers MainThread - WARNING:  Concatenating cubes in /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/rlut/gr/v20200420 failed, attempting workarounds
2021-07-19 05:44:42 netcdf_scm.iris_cube_wrappers MainThread - WARNING:  Removing `time_origin` attribute from all cubes
2021-07-19 05:44:42 netcdf_scm.iris_cube_wrappers MainThread - WARNING:  Forcing `long_name` of all cubes to be `TOA Outgoing Longwave Radiation`
2021-07-19 05:44:42 netcdf_scm.crunching MainThread - ERROR:  Could not calculate size of data in /data/cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3/piControl/r2i1p1f1/Amon/rlut/gr/v20200420, exception: failed to concatenate into a single cube.
  Dimension coordinates metadata differ: time != time
Traceback (most recent call last):
  File "/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages

Sorting directories: 100%|██████████| 613/613 [01:25<00:00,  7.16it/s]
2021-07-19 05:45:13 netcdf_scm.crunching MainThread - INFO:  Crunching 354 directories with less than 100 million data points
2021-07-19 05:45:13 netcdf_scm.cli_parallel MainThread - INFO:  Processing in parallel with 20 workers
2021-07-19 05:45:13 netcdf_scm.cli_parallel MainThread - INFO:  Forcing dask to use a single thread when reading
  0%|          | 0.00/354 [00:00<?, ?it/s]2021-07-19 05:45:14 netcdf_scm.crunching MainThread - INFO:  Attempting to process: ['rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_195101-196012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_201101-202012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_202101-203012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_207101-208012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_203101-204012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_192601-193512.nc', 'rlut_Amon_IITM-ESM_piControl_r1i1p1f1_gn_196101-197012.nc', 'rlut_Amon_IITM-ESM_piControl_r1i

2021-07-19 05:45:14 netcdf_scm.crunching MainThread - INFO:  Attempting to process: ['rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_185001-185912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_195001-195912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_193001-193912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_190001-190912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_192001-192912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_194001-194912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_191001-191912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_188001-188912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_198001-198912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_197001-197912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_200001-200912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_196001-196912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_199001-199912.nc', 'rsut_Amon_IITM-ESM_abrupt-4xCO2_r1i1p1f1_gn_201001-201412.nc', 'rsut_Amon_IITM-ESM

/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
2021-07-19 05:45:14 netcdf_scm.crunching MainThread - INFO:  Determining valid regions
2021-07-19 05:45:14 netcdf_scm.crunching MainThread - INFO:  Determining valid regions
2021-07-19 05:45:14 netcdf_scm.iris_cube_wrappers MainThread - WARNING:  Couldn't find/use areacell_cube, falling back to iris.analysis.cartography.area_weights
2021-07-19 05:45:14 netcdf_scm.iris_cube_wrappers MainThread - WARNING:  Couldn't find/use areacell_cube, falling back to iris.analysis.cartography.area_weights
/data/ubuntu-znicholls/miniconda3/envs/cmip6-country-level-processing/lib/python3.8/site-packages/iris/analysis/cartography.py:394: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")
